In [ ]:
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from cbc_paths import file_path_csv, chrome_executable_path, edge_executable_path

# create selenium driver
#driver = webdriver.Edge(executable_path=edge_executable_path)
driver = webdriver.Chrome(executable_path=chrome_executable_path)
driver.minimize_window()

# read json file data
file = os.path.abspath('') + '\cbc_stories.json'
with open(file) as json_file:
    cbc_stories = json.load(json_file)
stories = cbc_stories['cbc_stories']
print(len(stories), 'stories')

# loop through stories to get most recent data
story_count = 0
for story in stories:
    story_count += 1
    cbc_url = story['url']
    print(story_count, cbc_url)
    driver.get(cbc_url)
    time.sleep(5)

    # get csv record count
    file_name = cbc_url.replace('https://www.cbc.ca/','').replace('news/','').replace('sports/','').replace('/','_') + '.csv'
    try: # new urls may not yet have csv files
        with open(file_path_csv + file_name, encoding='utf-8') as f:
            comment_count_data = str(sum(1 for line in f))
        story_csv = 'yes'
    except: 
        comment_count_data = ''
        story_csv = 'no'

    # get cbc story data from selenium
    comment_count_page = driver.find_element(By.CLASS_NAME, 'vf-counter').text
    story_title = driver.find_element(By.CLASS_NAME, 'detailHeadline').text
    story_date = driver.find_element(By.XPATH, '//time').get_attribute('datetime').split('T')[0]
    
    # write updated data to json
    story['comment_count_page'] = comment_count_page
    story['story_title'] = story_title
    story['story_date'] = story_date
    story['csv'] = story_csv
    story['comment_count_data'] = comment_count_data
    try:
        driver.find_element(By.XPATH, "//*[contains(text(), 'Commenting is now closed for this story')]")
        story['comment_state_page'] = 'closed'
    except:
        comment_state = 'open'
        story['comment_state_page'] = 'open'

# save updated json to json file
with open(file, 'w') as file_object:
    json.dump(cbc_stories, file_object, indent=4)

# close browser
driver.close()